In [7]:
import cv2
import torch
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F

def gaussian_kernel(kernel_size, sigma):
    # Create a 1D Gaussian kernel
    kernel = cv2.getGaussianKernel(kernel_size, sigma)

    # Compute the outer product to get a 2D Gaussian kernel
    kernel = np.outer(kernel, kernel)

    return kernel

# Function to apply Gaussian blur with different sigma values
def apply_gaussian_blur(image_tensor, sigmas, device):
    blurred_images = []
    for sigma in sigmas:
        # Create a 2D Gaussian kernel
        gaussian_kernel_2d = gaussian_kernel(kernel_size, sigma)
        # Convert the kernel to a PyTorch tensor
        gaussian_kernel_2d = torch.tensor(gaussian_kernel_2d, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)
        # Apply the Gaussian blur using convolution
        tensor_blurred = F.conv2d(image_tensor, gaussian_kernel_2d, padding=kernel_size//2)[0]
        blurred_images.append(tensor_blurred.cpu().numpy()[0])
    return blurred_images

# Specify the device string
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('device: ', device)

# Load and preprocess the dataset
# Assuming you have a list of image file paths named 'image_paths'
main_directory = r'/bovwData/'
image_paths = [f'{main_directory}cat1.jpg', f'{main_directory}cat2.jpg', f'{main_directory}cat3.jpg', f'{main_directory}cat4.jpg', f'{main_directory}cat5.jpg', f'{main_directory}cat6.jpg', f'{main_directory}cat7.jpg', f'{main_directory}cat8.jpg', f'{main_directory}cat9.jpg', f'{main_directory}cat10.jpg',]  # Insert your image file paths here

# Load and preprocess each image in the dataset
images = []
for path in image_paths:
    img = Image.open(path)
    gray_img = img.convert('L')
    tensor_img = torch.tensor(np.array(gray_img), dtype=torch.float32).unsqueeze(0).unsqueeze(0) / 255.0
    tensor_img = tensor_img.to(device)
    images.append(tensor_img)

# Define kernel size
kernel_size = 50

# Define sigma values for Gaussian filters
sigmas = [1,2,3,4,5,6,7,8,9,10]

# Apply Gaussian blur to all images in the dataset with different sigma values
blurred_images = []
for image_tensor in images:
    blurred_images_per_image = apply_gaussian_blur(image_tensor, sigmas, device)
    blurred_images.append(blurred_images_per_image)

# Display the original and result images using matplotlib subplots
num_images = len(images)
num_sigmas = len(sigmas)

fig, axs = plt.subplots(num_images, num_sigmas + 1, figsize=(12, 6))

# Display the original and result images using separate matplotlib plots for each image
for i in range(num_images):
    fig, axs = plt.subplots(1, num_sigmas + 1, figsize=(12, 6))

    # Original image
    axs[0].imshow(images[i][0].cpu().numpy().squeeze(), cmap='gray')
    axs[0].set_title('Original Image')
    axs[0].axis('off')

    # Blurred images with different sigmas
    for j in range(num_sigmas):
        axs[j+1].imshow(blurred_images[i][j].squeeze(), cmap='gray')
        axs[j+1].set_title('Sigma = {}'.format(sigmas[j]))
        axs[j+1].axis('off')

    plt.tight_layout()
    plt.show(block=True)



device:  cpu


FileNotFoundError: [Errno 2] No such file or directory: '/bovwData/cat1.jpg'